In [ ]:
import sys
sys.path.insert(0, '..')

import torch.nn as nn
from torch.utils.data import DataLoader
from utils.eye_dataset import *
from eye_classifier import *
import torchvision.transforms as transforms
import torchvision.models as models

In [ ]:
base_dir = "../../data"
image_dir = f"{base_dir}/preprocessed_images"

image_dir_training = f"{base_dir}/ODIR-5K/training"
image_dir_testing = f"{base_dir}/ODIR-5K/testing"
csv_file = f'{base_dir}/ODIR-5K/data.csv'

print ('reading input dataset')
input_size = 224

apply_transforms = transforms.Compose([
    transforms.Resize(size=input_size),
    transforms.CenterCrop(size=input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ds = EyeImageDataset(root=image_dir_training, data_info_csv_file=csv_file, transform=apply_transforms)
train_loader = DataLoader(ds, batch_size=4, shuffle=True)


Building the model

In [ ]:
class ResnetEyeClassifier(EyeClassifier):
    def __init__(self, num_classes: int) -> None:
        super(ResnetEyeClassifier, self).__init__(model=[

            (models.resnet18(pretrained=True), TransferFunction.NotApplicable),

            (nn.Linear(in_features=1000, out_features=84),
             TransferFunction.Relu),

            (nn.Linear(in_features=84, out_features=42),
             TransferFunction.Relu),

            (nn.Linear(in_features=42, out_features=num_classes),
             TransferFunction.NotApplicable),
        ])


nn = ResnetEyeClassifier(num_classes=len(ds.classes))
print(nn)

Training the model

In [ ]:
nn.train(ds)


Testing the model with the training set

In [ ]:

nn.test(ds)

Testing the network with the test set

In [ ]:
ds_test = EyeImageDataset(root=image_dir_testing, data_info_csv_file=csv_file, tranform=tranform_method)
test_loader = DataLoader(ds_test, batch_size=4, shuffle=False)

nn.test(ds)